In [1]:
import numpy as np
#np.set_printoptions(precision=10)

# Randomly generated symmetric doubly stochastic matrices whose spectra sums up to $\delta \in [\frac{1}{2}, 1]$
## Authors: Michał Gnacik and Tomasz Kania

### Supplementary to:  Inverse problems for symmetric doubly stochastic matrices whose Suleuimanova spectra are to be bounded below by 1/2. 
### Arxiv link: <a href ="https://arxiv.org/abs/1909.01291">1909.01291 </a>

Let $\sigma = (1, \lambda_2, \ldots, \lambda_{n})$, where $\lambda_i \leq 0$ for $i = 2, \ldots, n$,
be such that
$$1 + \lambda_2 + \ldots + \lambda_n \geqslant \delta$$
for some $\delta >0$.

Below we present a simple algorithm how to generate doubly stochastic matrices such that the sum of $\sigma$ is $\geq \frac{1}{2}$. 

In [2]:
""" This function returns kth-row, lth-column element of a matrix"""
def matrix_entries(k,l, spectrum):
    # k and l: int, it should range (0, spectrum.size)
    # spectrum: np.array, spectrum sigma = (1, lambda_2, ..., lambda_n)
    # Output: float, returns the element of a matrix on the kth row and lth column. 
    n = spectrum.size
    A = [np.sin(2*np.pi*k*j / n + np.pi/4)*np.sin(2*np.pi*l*j / n + np.pi/4) for j in range(1,n)]
    return 1/n*(1+2*np.dot(spectrum[1:], A))

In [3]:
""" This function generate a random spectrium that sums up to sul_delta"""
def random_spectrum(n, sul_delta, distribution = "np.random.uniform"):
    # n: int, size of the spectrum
    # sul_delta: float, should be taken from  [0.5, 1].
    # distibution: any np (or scipy) distribution supported on (0, 1)
    # Output: np.array, returns the spectrum that sums up to sul_delts
    m = n-1
    alpha = sul_delta-1
    spectrum = eval(distribution)(0,1, size=m)
    spectrum = alpha*spectrum/np.sum(spectrum)
    return np.concatenate([[1], spectrum])

In [4]:
""" This function checks if the spectrum generates a valid symmetric doubly stochastic matrix"""
def verify_if_mat_valid(spectrum):
    # spectrum: np.array, spectrum sigma = (1, lambda_2, ..., lambda_n)
    # Output: bool, True or False depending if the spectrum 
    #         let us construct symmetric doubly stochastic matrix with matrix_entries
    m = spectrum[1::].size
    for k in range(m-1):
        for l in range(k, m-1):
            value = matrix_entries(k, l, spectrum[1::])
            if value<0.0:
                return False#spectrum.size+1,value,spectrum, k
    return True

In [5]:
""" This function constructs a symmetric doubly stochastic n x n matrix whose spectrum sums up to sul_delta """
def constr_ds_matrix(n, sul_delta):
    # n: int, size of the spectrum
    # sul_delta: float, should be taken from  [0.5, 1].
    # returns: np.array, a symmetric doubly stochastic matrix, with spectrume that sums up to sul_delta
    spectrum = random_spectrum(n, sul_delta)
    m = n-1
    assert verify_if_mat_valid(spectrum)
    P = np.array([[ matrix_entries(k, l, spectrum) for k in range(n)] for l in range(n)])
    return P

### Examples of symmetric double stochastic $n \times n$ matrix whose spectrum sums up to $\delta$

In [9]:
n = 5 #dimension of rows and column space of your matrix
sul_delta = 0.5 # the sum of the spectrum
A = constr_ds_matrix(n, sul_delta) # example of a symmetric double stochastic matrix
print(A)

[[0.1        0.15354037 0.26847547 0.28976443 0.18821973]
 [0.15354037 0.08935552 0.18152453 0.29645963 0.27911995]
 [0.26847547 0.18152453 0.11733968 0.17088005 0.26178027]
 [0.28976443 0.29645963 0.17088005 0.08266032 0.16023557]
 [0.18821973 0.27911995 0.26178027 0.16023557 0.11064448]]


# Conditions on spectra that lead to a symmetric doubly stochastic matrix

# Perfect-Mirsky condition
[H. Perfect, L. Mirsky, Spectral properties of doubly-stochastic matrices, Monatsh. Math. 69 (1965) 35-57.]

Let $n \in \mathbb{N}$ if
$$\frac{1}{n} +\frac{1}{n(n-1)}\lambda_2 + \ldots + \frac{\lambda_{n}}{2\cdot 1} \geqslant 0$$
then there exists a symmetric doubly stochastic matrix with spectrum $\sigma = (1, \lambda_2, \ldots, \lambda_{n})$.

In [24]:
""" This function verifies if Perfect-Mirsky condition is satisfied given the spectrum """
def per_mir(spectrum):
    # spectrum: np.array, spectrum sigma = (1, lambda_2, ..., lambda_n)
    spectrum = np.sort(spectrum)[::-1]
    n = spectrum.size
    formula = np.array([1/((n-k)*(n-k+1))*spectrum[1:][k-1] for k in range(1, n)])
    return 1/n + np.sum(formula)

# Soules condition
[G.W. Soules, Constructing symmetric nonnegative matrices, Linear Multilinear Algebra 13 (1983), 241-251.]

Let $m, n \in \mathbb{N}$ be such that $n=2m +1$ in the case $n$ is odd and $n=2m+2$ in the case where $m$ is even. If
\begin{equation}\label{eqn: soules}\frac{1}{n} +\frac{n-m-1}{n(m+1)}\lambda_2  + \sum_{k=1}^n\frac{\lambda_{n-2k+2}}{(k+1)k} \geqslant 0,\end{equation}
then there exists a symmetric doubly stochastic matrix with spectrum $\sigma = (1, \lambda_2, \ldots, \lambda_{n})$.

In [25]:
""" This function verifies if Soules condition is satisfied given the spectrum """
def soules(spectrum):
    # spectrum: np.array, spectrum sigma = (1, lambda_2, ..., lambda_n)
    spectrum = np.sort(spectrum)[::-1]
    n = spectrum.size
    if n%2 == 0:
        m = int((n-2)/2)
    else:
        m = int((n-1)/2)
    formula = np.array([1/(r*(r+1))*spectrum[1:][n-2*r] for r in range(1, m+1)])
    return 1/n+(n-m-1)/(n*(m+1))*spectrum[1:][0]+np.sum(formula)

### Spectra that sum up to 0.5 but do not satisfy Perfect-Mirsky or Soules conditions

In [26]:
sigma5 = np.array([1,-0.02, -0.03, -0.05, -0.4])
sigma6 = np.array([1, -0.01, -0.02, -0.06, -0.08, -0.33])

print(per_mir(sigma5))
print(per_mir(sigma6))
print(soules(sigma5))
print(soules(sigma6))

-0.011833333333333335
-0.018000000000000016
-0.007666666666666683
-0.010000000000000037


# New Condition 1 - Improved Soules condition (for n even)
Theorem 3, Notation 1, Observation 1 in [R. Nader, B. Mourad, A. Bretto, and H. Abbas, A note on the real inverse spectral problem for doubly stochastic matrices, Linear Algebra Appl, 569 (2019), 206--240].
Let $n \geq 5$ and let $n$ be even, if

$$\frac{1}{n} + \frac{1}{n}\lambda_2 + \frac{\frac{n}{2}- \left[\frac{n+2}{4} \right]}{\frac{n}{2}\left[\frac{n+2}{4} \right]}\lambda_4+\sum_{k=1}^{\left[\frac{n+2}{4} \right]-1}\frac{\lambda_{n-4k+4}}{k(k+1)} \geq 0$$
then there exists a symmetric doubly stochastic matrix with spectrum $\sigma = (1, \lambda_2, \ldots, \lambda_{n})$.

In [27]:
""" This function verifies if Improved Soules condition is satisfied given the spectrum """
def imp_soules(spectrum):
    # spectrum: np.array, spectrum sigma = (1, lambda_2, ..., lambda_n)
    spectrum = np.sort(spectrum)[::-1]
    n = spectrum.size
    assert n%2==0
    N = int(np.floor((n+2)/4))
    formula = np.array([1/(r*(r+1))*spectrum[1:][n-4*r+2] for r in range(1, N)])
    return 1/n + 1/n*spectrum[1:][0]+(n/2-N)/(n/2*N)*spectrum[1:][2]+np.sum(formula)

### Spectrum that sums up to 0.5 but does not satisfy Improved Soules cond. 

In [28]:
sigma20 = np.array([1, -0.01,  -0.01,  -0.025, -0.03,  -0.035, -0.04,  -0.05,  -0.08,  -0.22])
print(imp_soules(sigma20))

-0.020166666666666652


# New Condition 1 - Independent from the Soules Condition (for n odd)
Theorem 4, Notation 1 in [R. Nader, B. Mourad, A. Bretto, and H. Abbas, A note on the real inverse spectral problem for doubly stochastic matrices, Linear Algebra Appl, 569 (2019), 206--240]. Let $n \geq 5$ and let $n$ be odd, if

$$\frac{1}{n} + \frac{n-1}{n(n+1)}\lambda_2 + \frac{\frac{n+1}{2}- \left[\frac{n+3}{4} \right]}{\frac{n+1}{2}\left[\frac{n+3}{4} \right]}\lambda_4+\sum_{k=1}^{\left[\frac{n+3}{4} \right]-1}\frac{\lambda_{n-4k+4}}{k(k+1)} \geq 0$$
then there exists a symmetric doubly stochastic matrix with spectrum $\sigma = (1, \lambda_2, \ldots, \lambda_{n})$.

In [29]:
""" This function verifies if new condition 1 is satisfied given the spectrum """
def new_cond1(spectrum):
    # spectrum: np.array, spectrum sigma = (1, lambda_2, ..., lambda_n)
    spectrum = np.sort(spectrum)[::-1]
    n = spectrum.size
    assert n%2!=0
    N = int(np.floor((n+3)/4))
    formula = np.array([1/(r*(r+1))*spectrum[1:][n-4*r+2] for r in range(1, N)])
    return 1/n + (n-1)/(n*(n+1))*spectrum[1:][0]+((n+1)/2-N/((n+1)/2*N))*spectrum[1:][2]+np.sum(formula)

#### Spectrum that sums up to 0.5 but does not satisfy New Cond. 1. 

In [30]:
sigma5 = np.array([1,-0.03, -0.03, -0.04, -0.4])
print(new_cond1(sigma5))

-0.11066666666666666


# New Condition 2
Theorem 5, Notation 2 in [R. Nader, B. Mourad, A. Bretto, and H. Abbas, A note on the real inverse spectral problem for doubly stochastic matrices, Linear Algebra Appl, 569 (2019), 206--240]. Let $m \geq 2$. 

If $n=4m$ and

$$\frac{1}{n}+\frac{1}{n}\lambda_2+\frac{2}{n}\lambda_4 + \frac{\frac{n}{4} - \left[\frac{n+4}{8}\right]}{\frac{n}{4} \left[\frac{n+4}{8}\right]}\lambda_8 +\sum_{k=1}^{\left[\frac{n+4}{8}\right]-1}\frac{\lambda_{n-8k+8}}{k(k+1)} \geq 0,$$
or $n=4m+2$ and
$$\frac{1}{n}+\frac{1}{n}\lambda_2+\frac{2(n-2)}{n(n+2)}\lambda_4 + \frac{\frac{n+2}{4} - \left[\frac{n+6}{8}\right]}{\frac{n+2}{4} \left[\frac{n+6}{8}\right]}\lambda_8 +\sum_{k=1}^{\left[\frac{n+6}{8}\right]-1}\frac{\lambda_{n-8k+8}}{k(k+1)} \geq 0,$$
or $n=4m+3$ and
$$\frac{1}{n}+\frac{n-1}{n(n+1)}\lambda_2+\frac{2}{n+1}\lambda_4 + \frac{\frac{n+1}{4} - \left[\frac{n+5}{8}\right]}{\frac{n+1}{4} \left[\frac{n+5}{8}\right]}\lambda_8 +\sum_{k=1}^{\left[\frac{n+5}{8}\right]-1}\frac{\lambda_{n-8k+8}}{k(k+1)} \geq 0,$$
or $n=4m+1$ and
$$\frac{1}{n}+\frac{n-1}{n(n+1)}\lambda_2+\frac{2(n-1)}{(n+1)(n+3)}\lambda_4 + \frac{\frac{n+3}{4} - \left[\frac{n+7}{8}\right]}{\frac{n+3}{4} \left[\frac{n+7}{8}\right]}\lambda_8 +\sum_{k=1}^{\left[\frac{n+7}{8}\right]-1}\frac{\lambda_{n-8k+8}}{k(k+1)} \geq 0 $$
then there exists a symmetric doubly stochastic matrix with spectrum $\sigma = (1, \lambda_2, \ldots, \lambda_{n})$.

In [31]:
def last_sum(N, spectrum):
    # N: numerical value, int or float
    # spectrum: np.array, spectrum sigma = (1, lambda_2, ..., lambda_n)
    N = int(N)
    n = spectrum.size
    return np.sum(np.array([1/(r*(r+1))*spectrum[1:][n-8*r+6] for r in range(1, N)]))


""" This function verifies if new condition 2 is satisfied given the spectrum """
def new_cond2(spectrum):
    # spectrum: np.array, spectrum sigma = (1, lambda_2, ..., lambda_n)
    spectrum = np.sort(spectrum)[::-1]
    n = spectrum.size
    if n%4==0:
        N=np.floor((n+4)/8)
        return 1/n + 1/n*spectrum[1:][0]+2/n*spectrum[1:][2]*(n/4-N)/((n/4)*N)*spectrum[1:][6]+last_sum(N, spectrum)
    elif n%4==2:
        N=np.floor((n+6)/8)
        return 1/n + 1/n*spectrum[1:][0]+2*(n-2)/(n*(n+2))*spectrum[1:][2]*((n+2)/4-N)/(((n+2)/4)*N)*spectrum[1:][6]+last_sum(N, spectrum)
    elif n%4==3:
        N=np.floor((n+5)/8)
        return 1/n + (n-1)/(n*(n+1))*spectrum[1:][0]+2/(n+1)*spectrum[1:][2]*((n+1)/4-N)/(((n+1)/4)*N)*spectrum[1:][6]+last_sum(N, spectrum)
    else:
        N=np.floor((n+7)/8)
        return 1/n + (n-1)/(n*(n+1))*spectrum[1:][0]+2*(n-1)/((n+1)*(n+3))*spectrum[1:][2]*((n+3)/4-N)/(((n+3)/4)*N)*spectrum[1:][6]+last_sum(N, spectrum)

#### Spectra that sums up to 0.5 but does not satisfy New Cond. 2. 

In [32]:
sigma16 = np.array([1, -0.003,  -0.003,  -0.004,  -0.007,  -0.009,  -0.02,   -0.0209,-0.021,  -0.024,
 -0.026,  -0.035,  -0.042,  -0.076,  -0.0811, -0.128])
print(new_cond2(sigma16))

sigma10 = np.array([1, -0.01, -0.01, -0.01, -0.02, -0.02, -0.04, -0.07, -0.1,-0.22])
print(new_cond2(sigma10))

sigma11 = np.array([1, -0.001, -0.004, -0.01,  -0.01,  -0.012, -0.013, -0.05,  -0.09,  -0.11,  -0.2 ])
print(new_cond2(sigma11))

sigma9 = np.array([1, -0.006, -0.018, -0.02,  -0.028, -0.028, -0.053, -0.105, -0.242])
print(new_cond2(sigma9))

-0.0016848875000000027
-0.010984444444444436
-0.009152777777777787
-0.010375555555555555


# New Condition 3,  the case when $n =26$
Conjecture 1, Example 1 in [R. Nader, B. Mourad, A. Bretto, and H. Abbas, A note on the real inverse spectral problem for doubly stochastic matrices, Linear Algebra Appl, 569 (2019), 206--240]. If


$$\frac{1}{26}+\frac{1}{26}\lambda_2 + \frac{6}{13\cdot 7}\lambda_4 + \frac{3}{28}\lambda_8+\frac{1}{4}\lambda_{16}+\frac{1}{2}\lambda_{26}\geq 0$$
then there exists a symmetric doubly stochastic matrix with spectrum $\sigma = (1, \lambda_2, \ldots, \lambda_{26})$.

In [33]:
""" This function verifies if new condition 3 is satisfied given the spectrum """
def new_cond3(spectrum):
    # spectrum: np.array, spectrum sigma = (1, lambda_2, ..., lambda_n)
    spectrum = np.sort(spectrum)[::-1]
    n = spectrum.size
    assert n==26
    return 1/n+1/n*spectrum[1:][0]+(12)/(n*7)*spectrum[1:][2]+3/(n+2)*spectrum[1:][6]+1/4*spectrum[1:][14]+1/2*spectrum[1:][24]

#### Spectrum that sums up to 0.5 but does not satisfy New Cond. 3. 

In [34]:
sigma26 = [1, -0.004, -0.005, -0.006, -0.007, -0.01,  -0.01,  -0.011, -0.011, -0.011, -0.012,
 -0.012, -0.015, -0.015, -0.016, -0.017, -0.019, -0.02,  -0.022, -0.022, -0.025,
 -0.028, -0.028, -0.032, -0.069, -0.073]
print(new_cond3(sigma26))

-0.004016483516483517


# Generating examples of spectra that sum up to $\delta \in [\frac{1}{2}, 1]$
# but do not satisfy the above conditions. 

### Perfect-Mirsky and Soules conditions

In [67]:
def generate_examples_pms(A, B, sul_delta, trials = 1000):
    # A: int, to start the for loop with a spectrum with A elements
    # B: int, to end  the for loop with a spectrum with B elements
    # sul_delta: float, should be taken from  [0.5, 1].
    # trials: int, number of different spectra generated. 
    # Output: returns two lists, 
    #         first list contains examples when Perfect-Mirsky condition does not hold
    #         second list contains examples when Soules condition does not hold
    results_per_mir = []
    results_soules = []
    for n in range(A, B):
        for i in range(trials):
            spectrum = random_spectrum(n, sul_delta)
            spectrum = np.sort(spectrum)[::-1]
            if per_mir(spectrum)<0:
                results_per_mir.append(spectrum)
                #print("Perfect-Mirsky cond.", spectrum, per_mir(spectrum))
            if soules(spectrum)<0:
                results_soules.append(spectrum)
                #print("Soules cond.", spectrum, soules(spectrum)
    return results_per_mir, results_soules
    

In [105]:
A, B = 5, 7
sul_delta = 0.5
trials = 50
print(generate_examples_pms(A, B, sul_delta, trials)[0])
print("###########################")
print(generate_examples_pms(A, B, sul_delta, trials)[0])

[array([ 1.        , -0.00305868, -0.00549608, -0.0306248 , -0.17555056,
       -0.28526989])]
###########################
[array([ 1.        , -0.00347896, -0.04957362, -0.06189354, -0.38505388])]


### Improved condition 1 (contains Improved Soules condition for n even)

In [86]:
def generate_examples_imp_cond1(A, B, sul_delta, trials = 1000):
    # A: int, to start the for loop with a spectrum with A elements
    # B: int, to end  the for loop with a spectrum with B elements
    # sul_delta: float, should be taken from  [0.5, 1].
    # trials: int, number of different spectra generated. 
    # Output: returns two lists, 
    #         first list contains examples when Improved condition 1 does not hold (n odd)
    #         second list contains examples when improved Soules condition does not hold (n even)
    results_cond1 = []
    results_imp_soules = []
    assert A>=5
    for n in range(A, B):
        for i in range(trials):
            spectrum = random_spectrum(n, sul_delta)
            spectrum = np.sort(spectrum)[::-1]
            if n%2==0:
                if imp_soules(spectrum)<0:
                    results_imp_soules.append(spectrum)
                    #print("Improved Soulesd cond.", spectrum, imp_soules(spectrum))
            if n%2!=0:
                if new_cond1(spectrum)<0:
                    results_cond1.append(spectrum)
                    #print("Soules cond.", spectrum, soules(spectrum)
    return results_cond1, results_imp_soules

In [108]:
sul_delta = 0.5
A, B = 5, 6
trials = 10
print(generate_examples_imp_cond1(A, B, sul_delta, trials)[0])
A, B = 6, 7
trials = 100
print("###########################")
print(generate_examples_imp_cond1(A, B, sul_delta, trials)[1])

[array([ 1.        , -0.07466115, -0.1012881 , -0.15181129, -0.17223946]), array([ 1.        , -0.07846531, -0.10576005, -0.15424814, -0.1615265 ]), array([ 1.        , -0.06344029, -0.09099451, -0.16943802, -0.17612717]), array([ 1.        , -0.06481122, -0.10508408, -0.1109672 , -0.21913751]), array([ 1.        , -0.04710657, -0.12321536, -0.13382162, -0.19585645]), array([ 1.        , -0.07437103, -0.10842734, -0.13013879, -0.18706284]), array([ 1.        , -0.03379112, -0.09340607, -0.1217603 , -0.25104251]), array([ 1.        , -0.0213104 , -0.02187269, -0.17577125, -0.28104566]), array([ 1.        , -0.05443527, -0.06320256, -0.16466425, -0.21769792]), array([ 1.        , -0.06335596, -0.11268071, -0.15209612, -0.17186721])]
###########################
[array([ 1.00000000e+00, -1.22505120e-04, -1.18958058e-02, -5.52536140e-02,
       -8.28875675e-02, -3.49840508e-01])]


### Improved condition 2 

In [111]:
def generate_examples_imp_cond2(A, B, sul_delta, trials = 1000):
    # A: int, to start the for loop with a spectrum with A elements
    # B: int, to end  the for loop with a spectrum with B elements
    # sul_delta: float, should be taken from  [0.5, 1].
    # trials: int, number of different spectra generated. 
    # Output: returns a list of spectra that does not satisfy impoved condition 2. 
    results_cond2=[]
    assert A>=8
    for n in range(A, B):
        for i in range(trials):
            spectrum = random_spectrum(n, sul_delta)
            spectrum = np.sort(spectrum)[::-1]
            if new_cond2(spectrum) <0:
                results_cond2.append(spectrum)
    return results_cond2

In [123]:
sul_delta = 0.5
A, B = 16, 17
trials = 5000
print(generate_examples_imp_cond2(A, B, sul_delta, trials))
A, B = 9, 10
trials = 5000
print("###########################")
print(generate_examples_imp_cond2(A, B, sul_delta, trials))
A, B = 10, 11
trials = 5000
print("###########################")
print(generate_examples_imp_cond2(A, B, sul_delta, trials))
A, B = 11, 12
trials = 5000
print("###########################")
print(generate_examples_imp_cond2(A, B, sul_delta, trials))

[array([ 1.        , -0.00194477, -0.00422638, -0.00481714, -0.0051333 ,
       -0.00915552, -0.01205872, -0.01748905, -0.02643285, -0.02695055,
       -0.03454564, -0.03663943, -0.04256653, -0.06001726, -0.09296968,
       -0.12505319])]
###########################
[array([ 1.        , -0.01773602, -0.02974931, -0.03308642, -0.03464276,
       -0.04393104, -0.04464775, -0.07675685, -0.21944985]), array([ 1.        , -0.00442101, -0.00677522, -0.01095098, -0.01526127,
       -0.02539361, -0.04796391, -0.07875687, -0.31047713]), array([ 1.        , -0.00647115, -0.0122118 , -0.01845398, -0.02211989,
       -0.03392391, -0.07621437, -0.10863061, -0.22197429])]
###########################
[array([ 1.00000000e+00, -3.40549956e-04, -1.16692407e-03, -1.53311385e-02,
       -2.85901802e-02, -3.36993218e-02, -4.08988979e-02, -5.41962281e-02,
       -1.05766044e-01, -2.20010716e-01]), array([ 1.        , -0.00503086, -0.01576794, -0.02436544, -0.03410192,
       -0.03641722, -0.0367989 , -0.050

### Improved condition 3

In [110]:
def generate_examples_imp_cond3(A, B, sul_delta, trials = 1000):
    # A: int, to start the for loop with a spectrum with A elements
    # B: int, to end  the for loop with a spectrum with B elements
    # sul_delta: float, should be taken from  [0.5, 1].
    # trials: int, number of different spectra generated. 
    # Output: returns a list of spectra that does not satisfy impoved condition 3. 
    results_cond3=[]
    assert (A==26 and B==27)
    for n in range(A, B):
        for i in range(trials):
            spectrum = random_spectrum(n, sul_delta)
            spectrum = np.sort(spectrum)[::-1]
            if  new_cond3(spectrum) <0:
                results_cond3.append(spectrum)
    return results_cond3

In [128]:
sul_delta = 0.5
A, B = 26, 27
trials = 50000
print(generate_examples_imp_cond3(A, B, sul_delta, trials))

[array([ 1.00000000e+00, -8.54943138e-04, -1.63849066e-03, -1.99941602e-03,
       -2.98617509e-03, -3.00989912e-03, -5.85041849e-03, -9.29657170e-03,
       -9.30664389e-03, -9.51879274e-03, -1.01306938e-02, -1.26104731e-02,
       -1.37967053e-02, -1.45585407e-02, -1.75622069e-02, -1.82214551e-02,
       -2.40008730e-02, -2.71986721e-02, -2.78463380e-02, -2.92366050e-02,
       -3.14012729e-02, -3.45548818e-02, -3.52587079e-02, -4.54819300e-02,
       -4.66583062e-02, -6.70209873e-02])]
